## 1. Validation
#### File format validation and check for compulsory columns and total number of columns.

In [58]:
import os
import shutil
import pandas as pd
from dateutil.parser import parse


# Define the input and output directories
input_directory = r'D:\University of Bath\Assignments\Applied Data Science\Task 3\csv_data_2023'
output_directory = r'D:\University of Bath\Assignments\Applied Data Science\Task 3\Validated_data_2023'

# Remove the entire output directory
shutil.rmtree(output_directory, ignore_errors=True)

# Recreate the empty output directory
os.makedirs(output_directory, exist_ok=True)

# List the CSV files in the input directory
csv_files = [f for f in os.listdir(input_directory) if f.endswith('.csv')]

# Define the desired columns (Date, Time, Direction, Vehicle Type, Occupancy)
desired_columns = ['date', 'time', 'direction', 'type', 'occupancy']


# Initializing list of skipped files
skipped_files = []

# Initializing list of files with exception
files_with_exception = []


# Dictionary with unique values in type column
type_value_counts = {}



# Standardizing vehicle types
def standardize_vehicle_types(vehicle_type):
    if vehicle_type in ('car','bus','bicycle','taxi','van','motorbike','lorry','scooter'):
        return vehicle_type
    elif vehicle_type in ('mini truck', 'truck'):
        return 'lorry'
    elif vehicle_type in ('bike','motorcycle'):
        return 'motorbike'
    else:
        return 'other' # Others/Unknown values 


# Standardizing Occupancy
def standardize_occupancy(occupancy_value):
    try:
        return float(occupancy_value)
    except ValueError:
        percentage=float(occupancy_value.strip().rstrip('%'))
        return round((percentage/100)*(95))

# Iterate through each CSV file in the input directory
for file in csv_files:

    try:    
        # Read the CSV file
        df = pd.read_csv(os.path.join(input_directory, file),encoding='utf-8')
        df.columns = df.columns.str.lower().str.strip()
        
        # Validate that the required columns exist in the DataFrame
        all_columns_present = True

        #print(df.columns)
        for col in desired_columns:
            if col.strip().lower() not in df.columns:
                all_columns_present = False
                break
        
        # Check if all desired columns exist in the DataFrame
        if (not all_columns_present) or (len(df.columns)!=7):
            #print(f"Skipping {file} due to missing columns")
            skipped_files.append(file)
            continue



        for col in df.columns:
            
            # Convert 'Date' to a specific format
            
            if col.strip().lower() == 'date':
                transformation = False
                #print("date before: ", df[col][0])
                if transformation == False:
                    try:
                        df['Parsed_Date'] = df[col].apply(lambda x: parse(x, fuzzy=True, dayfirst=True))
                        #print(" Parsed date: ", df['Parsed_Date'][0], flush=True)
                        df[col] = df['Parsed_Date'].dt.strftime('%d-%m-%Y')
                        df.drop('Parsed_Date', axis=1, inplace=True)
                        transformation =  True
                        #print(transformation)
                    except Exception as x:
                        pass
                        #print(transformation)
                if transformation == False:
                    try:
                        df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)  # 'coerce' handles invalid dates
                        df[col] = df[col].dt.strftime('%d-%m-%Y')
                        transformation =  True
                        #print(transformation)
                    except Exception as e:
                        pass
                #print("date after: ", df[col][0])
                
            # Covert 'Time' to a specific format
            
            if col.strip().lower() == 'time':
                #print("Time before: ",df[col][0])
                df[col] = df[col].str.replace(r'am', '', case=False).str.strip()
                #print("Time without am/pm", df[col][0])
                x=df[col][0]

                time_format_boolean = False
                
                
                if df[col].str.contains('.', regex=False).any():
                    df[col] = pd.to_datetime(df[col], format='%I.%M', errors='coerce')
                    df[col] = df[col].dt.strftime('%H:%M:%S')
                    time_format_boolean = True
                    #print("Time after: ",df[col][0])
                
                if (df[col].str.contains(':', regex=False).any()) and (not time_format_boolean):
                    colon_count = df[col].str.count(':').max()
                    
                    if colon_count == 1:
                        time_format = '%H:%M'  # Format for '10:35am' or '10:35pm'
                    else:
                        time_format = '%H:%M:%S' 
                    
                    df[col] = pd.to_datetime(df[col],format=time_format, errors='coerce')
                    df[col] = df[col].dt.strftime('%H:%M:%S')
                    #print("Time after: ",df[col][0])               
            
            # Convert 'Direction' to a specific format
            if col.strip().lower() == 'direction':
                #print("Before:",df[col][0])
                df[col] = df[col].str.lower().str.strip()
                #print("After:",df[col][0])
            
            # Convert 'Type' to a specific format
            if col.strip().lower() == 'type':
                df[col] = df[col].str.lower().str.strip()
                df[col] = df[col].apply(standardize_vehicle_types)

            # Convert 'Occupancy' to a specific format
            if col.strip().lower() == 'occupancy':
                df[col] = df[col].apply(standardize_occupancy)

                        
            # You can add similar transformations for other desired columns here

        # Specify the output file path
        output_file = os.path.join(output_directory, f'transformed_{file}')
        
        # Save the DataFrame with transformed and additional columns
        df.to_csv(output_file, index=False)

        #print(f"Transformed and saved {file} to {output_file}")
    except Exception as e:
        print("File Unwell: ", file," has an error: ",e)
        files_with_exception.append(file)

#for key, value in type_value_counts.items():
#    print(str(key)+":"+str(value))    

print("Data transformation and validation complete.")
print("Skipped Files: ", len(skipped_files))
print("Exception Files: ", len(files_with_exception))


File Unwell:  22792751.csv  has an error:  'utf-8' codec can't decode byte 0xa0 in position 20: invalid start byte
Data transformation and validation complete.
Skipped Files:  16
Exception Files:  1


C:\Users\Vishwesh\AppData\Local\Temp\ipykernel_10972\1172296935.py:101: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)  # 'coerce' handles invalid dates


## 2. Additional Columns
#### Checking and grouping columns included in additional columns.

In [59]:
input_directory = r'D:\University of Bath\Assignments\Applied Data Science\Task 3\Validated_data_2023'

# Initialize a dictionary to store column occurrences
column_occurrences = {}

# List the CSV files in the input directory
csv_files = [f for f in os.listdir(input_directory) if f.endswith('.csv')]

# Iterate through each CSV file in the input directory
for file in csv_files:

    try:    
        # Read the CSV file
        df = pd.read_csv(os.path.join(input_directory, file),encoding='utf-8')
        df.columns = df.columns.str.lower().str.strip()

        # Extract the last two columns
        last_two_columns = df.iloc[:, -2:].columns.tolist()

        # Update the dictionary with column occurrences
        for column in last_two_columns:
            if column in column_occurrences:
                column_occurrences[column].append(file)
            else:
                column_occurrences[column] = [file]

                
    except Exception as e:
        print('File: ', file, 'has error: ', e)

# Print the dictionary of column occurrences
for column, files in column_occurrences.items():
    print(f"{column}: {len(files)} files - {files}")



vehicle age(years): 2 files - ['transformed_22792701.csv', 'transformed_22792711.csv']
brand name: 2 files - ['transformed_22792701.csv', 'transformed_22792711.csv']
additionaldata1: 1 files - ['transformed_22792702.csv']
additionaldata2: 7 files - ['transformed_22792702.csv', 'transformed_22792721.csv', 'transformed_22792742.csv', 'transformed_22792767.csv', 'transformed_22792780.csv', 'transformed_22792788.csv', 'transformed_22792795.csv']
color: 2 files - ['transformed_22792703.csv', 'transformed_22792803.csv']
public/private: 1 files - ['transformed_22792703.csv']
colour: 10 files - ['transformed_22792704.csv', 'transformed_22792708.csv', 'transformed_22792725.csv', 'transformed_22792732.csv', 'transformed_22792747.csv', 'transformed_22792754.csv', 'transformed_22792762.csv', 'transformed_22792785.csv', 'transformed_22792805.csv', 'transformed_22792806.csv']
turn signal: 2 files - ['transformed_22792704.csv', 'transformed_22792785.csv']
carpool: 1 files - ['transformed_22792706.csv

## 3. Additional Column Data Transformation and Segregation

In [60]:
input_root_directory = r'D:\University of Bath\Assignments\Applied Data Science\Task 3\Validated_data_2023'
output_root_directory = r'D:\University of Bath\Assignments\Applied Data Science\Task 3'

# Encoding into 1s and 0s
def standardize_electric_column(x):
    x['electric'] = x['electric'].astype(str).str.strip().str.lower().replace({'yes': 1, 'ev': 1, '1': 1, 'no': 0, 'non ev': 0, '0': 0})
    return x

# Create a mapping dictionary for column names
column_mapping = {'electric': ['electric', 'ev']}

# Remove the entire output directory
for standard_name, variations in column_mapping.items():
    directory = os.path.join(output_root_directory, standard_name)
    shutil.rmtree(directory, ignore_errors=True)

# List the CSV files in the input directory
csv_files = [f for f in os.listdir(input_root_directory) if f.endswith('.csv')]

# Iterate through each CSV file in the input directory
for file in csv_files:
    try:    
        # Read the CSV file
        df = pd.read_csv(os.path.join(input_root_directory, file),encoding='utf-8')
        df.columns = df.columns.str.lower().str.strip()
        
        # Iterate through the mapping dictionary
        for standard_name, variations in column_mapping.items():
            matching_columns = []
            for variation in variations:
                if variation.lower() in df.columns:
                    matching_columns.append(variation)

            if matching_columns:
                # Rename the matching columns to the standard name
                df.rename(columns={col: 'electric' for col in matching_columns}, inplace=True)

                # Select only the desired columns
                desired_columns = ['date', 'time', 'direction', 'type', 'occupancy', 'electric']
                df = df[desired_columns]
                
                # Apply the standardization function
                df = standardize_electric_column(df)
                
                # Create the output directory based on the standard name
                output_directory = os.path.join(output_root_directory, standard_name)
                os.makedirs(output_directory, exist_ok=True)

                # Copy the DataFrame to the output directory with the standardized column name
                output_file = os.path.join(output_directory, f"{standard_name}_{file}")
                df.to_csv(output_file, index=False)

        
        

    except Exception as e:
        print("File Unwell: ", file," has an error: ",e)

# Print a message indicating the process is complete
print("Files copied and column names standardized.")



Files copied and column names standardized.


## 4. Handling Data Duplication

In [61]:
input_root_directory = r'D:\University of Bath\Assignments\Applied Data Science\Task 3\electric'
output_root_directory = r'D:\University of Bath\Assignments\Applied Data Science\Task 3'


# List the CSV files in the input directory
csv_files = [f for f in os.listdir(input_root_directory) if f.endswith('.csv')]

# Dictionary to store the number of records for each date and time
records_count_list = []

# Iterate through each CSV file in the input directory
for file in csv_files:
    try:
        # Read the CSV file
        df = pd.read_csv(os.path.join(input_root_directory, file), encoding='utf-8')
        df.columns = df.columns.str.lower().str.strip()

        # Group by date and time and count the number of records
        grouped_df = df.groupby(['date', 'time']).size().reset_index(name='records_count')

        # Dictionary to store records for the current file
        file_dict = {'file': file}

        # Update the file_dict with the maximum records for each date and time
        for index, row in grouped_df.iterrows():
            key = (row['date'], row['time'])
            file_dict[key] = row['records_count']

        # Append the file_dict to the records_count_list
        records_count_list.append(file_dict)

    except Exception as e:
        print("File Unwell: ", file, " has an error: ", e)


            
# Print the dictionary
for i in records_count_list:
    for key, value in i.items():
        print(key,': ', value)

file :  electric_transformed_22792719.csv
('24-10-2023', '09:40:00') :  3
('24-10-2023', '09:45:00') :  6
('24-10-2023', '09:50:00') :  13
('24-10-2023', '09:55:00') :  10
('24-10-2023', '10:00:00') :  9
('24-10-2023', '10:05:00') :  10
('24-10-2023', '10:10:00') :  6
file :  electric_transformed_22792720.csv
('24-10-2023', '10:30:00') :  14
('24-10-2023', '10:35:00') :  10
('24-10-2023', '10:40:00') :  4
('24-10-2023', '10:45:00') :  5
('24-10-2023', '10:50:00') :  7
('24-10-2023', '10:55:00') :  9
file :  electric_transformed_22792749.csv
('25-10-2023', '09:30:00') :  10
('25-10-2023', '09:35:00') :  5
('25-10-2023', '09:40:00') :  12
('25-10-2023', '09:45:00') :  6
('25-10-2023', '09:50:00') :  9
('25-10-2023', '09:55:00') :  13
file :  electric_transformed_22792755.csv
('24-10-2023', '10:30:00') :  32
('24-10-2023', '10:35:00') :  16
('24-10-2023', '10:40:00') :  19
('24-10-2023', '10:45:00') :  29
('24-10-2023', '10:50:00') :  26
('24-10-2023', '10:55:00') :  31
file :  electric_t

In [62]:
# Dictionary to store the file with maximum records for each date-time pair
max_records_files = {}

# Iterate through the list of dictionaries
for data_dict in records_count_list:
    # Iterate through the keys of each dictionary
    for key in data_dict.keys():
        # Check if the key is a tuple (date-time pair)
        if isinstance(key, tuple):
            # Check if this file has more records than the current maximum for the date-time pair
            if key not in max_records_files or data_dict[key] > max_records_files[key]['records']:
                max_records_files[key] = {'file': data_dict['file'], 'records': data_dict[key]}


# Print the unique date-time pairs
for key,value in max_records_files.items():
    print(key, ': ', value)

('24-10-2023', '09:40:00') :  {'file': 'electric_transformed_22792765.csv', 'records': 10}
('24-10-2023', '09:45:00') :  {'file': 'electric_transformed_22792765.csv', 'records': 18}
('24-10-2023', '09:50:00') :  {'file': 'electric_transformed_22792765.csv', 'records': 22}
('24-10-2023', '09:55:00') :  {'file': 'electric_transformed_22792765.csv', 'records': 30}
('24-10-2023', '10:00:00') :  {'file': 'electric_transformed_22792765.csv', 'records': 23}
('24-10-2023', '10:05:00') :  {'file': 'electric_transformed_22792765.csv', 'records': 43}
('24-10-2023', '10:10:00') :  {'file': 'electric_transformed_22792765.csv', 'records': 17}
('24-10-2023', '10:30:00') :  {'file': 'electric_transformed_22792755.csv', 'records': 32}
('24-10-2023', '10:35:00') :  {'file': 'electric_transformed_22792755.csv', 'records': 16}
('24-10-2023', '10:40:00') :  {'file': 'electric_transformed_22792755.csv', 'records': 19}
('24-10-2023', '10:45:00') :  {'file': 'electric_transformed_22792755.csv', 'records': 29}

In [63]:
input_root_directory = r'D:\University of Bath\Assignments\Applied Data Science\Task 3\electric'
output_root_directory = r'D:\University of Bath\Assignments\Applied Data Science\Task 3'

# Remove the entire output directory
directory = os.path.join(input_root_directory, 'electric_consolidated')
shutil.rmtree(directory, ignore_errors=True)

# Create an empty DataFrame to store consolidated records
consolidated_df = pd.DataFrame()

# Iterate through the keys of the max_records_files dictionary
for date_time_pair, file_info in max_records_files.items():
    # Extract the file with maximum records for the date-time pair
    file_path = os.path.join(input_root_directory, file_info['file'])
    
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Extract records for the specific date-time pair
    date, time = date_time_pair
    records_for_date_time_pair = df[(df['date'] == date) & (df['time'] == time)]
    
    # Append the extracted records to the consolidated DataFrame
    consolidated_df = pd.concat([consolidated_df, records_for_date_time_pair], ignore_index=True)

# Specify the output directory for the consolidated records
output_directory = os.path.join(output_root_directory, 'electric', 'electric_consolidated')

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Specify the output file path for the consolidated records
output_file_path = os.path.join(output_directory, 'electric_consolidated.csv')

# Save the consolidated DataFrame to a CSV file
consolidated_df.to_csv(output_file_path, index=False)

print(f"Consolidated records saved to {output_file_path}")

Consolidated records saved to D:\University of Bath\Assignments\Applied Data Science\Task 3\electric\electric_consolidated\electric_consolidated.csv
